## Deep learning

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


import data_processing
import feature_extraction

Load Data

In [2]:
feature_df = pd.read_csv("mode_classification_features.csv")
X = feature_df.drop(columns=['trajectory', 'label'])  # Drop irrelevant columns and the target.
Y = feature_df['label']  # Target variable

One-hot-transform

In [3]:
y_one_hot = pd.get_dummies(Y)
y_one_hot

,bike,bus,car,subway,taxi,train,walk
0,False,False,False,False,False,True,False
1,False,False,False,False,True,False,False
2,False,False,False,False,False,True,False
3,False,False,False,False,False,True,False
4,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...
3637,True,False,False,False,False,False,False
3638,False,False,False,False,False,False,True
3639,False,False,False,False,False,False,True
3640,False,False,False,False,False,False,True


Train/Test Split

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

Initialize the Model

In [5]:
def create_model(activation='relu', optimizer='adam', dropout_rate=0.3, neurons=128):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(X_train.shape[1],)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(int(neurons / 2), activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(7, activation='softmax'))  # 7 classes
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

Create the model

In [6]:
model = create_model()

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Train the model

In [7]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=50, 
                    batch_size=32, 
                    validation_data=(X_test, y_test),
                    verbose=1)


Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4668 - loss: 1.5353 - val_accuracy: 0.6612 - val_loss: 1.0729
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - accuracy: 0.6599 - loss: 1.0647 - val_accuracy: 0.6790 - val_loss: 0.9386
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - accuracy: 0.7137 - loss: 0.9019 - val_accuracy: 0.7064 - val_loss: 0.8769
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.7168 - loss: 0.8539 - val_accuracy: 0.7174 - val_loss: 0.8439
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7521 - loss: 0.7790 - val_accuracy: 0.7353 - val_loss: 0.8202
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 918us/step - accuracy: 0.7565 - loss: 0.7670 - val_accuracy: 0.7476 - val_loss: 0.7813
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - accuracy: 0.7706 - loss: 0.7052 - val_accuracy: 0.7613 - val_loss: 0.7486
Epoch 8/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7719 - loss: 0.7115 - val_accuracy: 0.7517 -

Evaluate on the test set

In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8046 - loss: 0.5823 
Test Loss: 0.5723450183868408
Test Accuracy: 0.8299039602279663


Plot

In [16]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

AttributeError: 'KerasClassifier' object has no attribute 'history'

### Cross Validation

In [11]:
model = KerasClassifier(model=create_model, verbose=0, activation='relu', dropout_rate=0.2, neurons=128, optimizer='adam')

In [15]:
history=model.fit(X_train, y_train, validation_data=(X_test, y_test))
history_dict = history.history_

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
